I need help for SIR model & SEIR model.
I am a student who have studied about Python since 2020.01
I am not uset to use Scipy but if I want to solve Parameter for SIR, SEIR model, I have to use Annealing method from Scipy. if u interested in epidemic disease, please Help me using my code. 

2주동안 중국에서 나온 논문을 참고하면서 SEIR 모델을 공부하였습니다. 코로나바이러스에 의해 분석관련 교육을 받고 있었는데 기간미정 교육이 미뤄지고(공공기관-정부지침여파) 이참에 코로나바이러스 관련 전염병 역학모델에 의거해서 예측모델 SIR, SEIR 모델을 공부하고 있는데, 많은 논문을 읽었지만, 제 수준에는 버거워서, 이 코드를 보시는 분들께 관심이 있으시다면 도움을 받고 싶네요.

beta 파라미터를 정확하게 구해야만 SEIR 모델이 완성될것같습니다.
2/18 31번 확진자에 의한 지역감염 당시의 모델을 그려서 현재 3주정도 지난 시점에서 실제비율과 비교해보고 싶습니다.

논문출처 :http://livecorona.co.kr/
lancet 논문 'Nowcasting and forecasting the potential domestic and international spread of the 2019-nCoV outbreak originating in Wuhan, China: a modelling study'  SEIR 모델 연구논문. 2020.1.25 발표

In [ ]:
# I need the parameter = Beta
# also Want to Know Ro:Reproductive number

# 1. 간단한 SIR 전염병 모델
# **simple SIR model epidemic without births or deaths**


![대체 텍스트](http://homepages.warwick.ac.uk/~masfz/ModelingInfectiousDiseases/Chapter2/Program_2.1/Equations.gif)

    β:	is the transmission rate and incorporates the encounter rate between susceptible and infectious individuals together with the probability of transmission.
    γ:	is called the **removal or recovery rate, though often we are more interested in its reciprocal (1/γ) which determines the average infectious period.
    S(0):	is the **initial proportion of the population that are susceptible.
    I(0):	is the **initial proportion of the population that are infectious.
All rates are specified in days.

**Requirements.**
All parameters must be positive, and S(0)+I(0) ≤ 1

In [ ]:
import scipy.integrate as spi
import numpy as np
import pylab as pl

beta =  0.1934 # S ->I 감염율 = beta 를 구하지 못해서 논문에 나온 beta 값 참조 ( 2020.02.10 중국논문발표)
#우리나라도 중국과 마찬가지로 2차 지역감염이 시작되고 전염력이 2/18 당시 매우 높았던것으로 보여 논문의 beta 도입.

gamma =  1/14 # I ->R 회복율 = 평균 회복기간의 역수
t_inc = 1.0 
t_end = 150.0

# 2/18 31번확진자 나온날 기준으로 초기 SIR 모델 만듦.
S0  = 9772 ; I0  = 31;R0 = 2

N = S0 + I0 + R0 
S0  = 9772 /N  # susceptible hosts
I0  = 31 /N    # infectious hosts
R0 = 2 /N      # recovered hosts

Input = (S0, I0, 0.0)

Input


 ![대체 텍스트](https://wikimedia.org/api/rest_v1/media/math/render/svg/d74226f805afb1847763d4588774981120389b85)

  ![대체 텍스트](https://wikimedia.org/api/rest_v1/media/math/render/svg/29dbe54a80c1f7ad16fc4553aef04d72b95dcc90)

  ![대체 텍스트](https://wikimedia.org/api/rest_v1/media/math/render/svg/64dd5e5d9472d7b194878f13019440e144f305cf)

In [ ]:
def simple_SIR(INT, t):
  '''The main set of equation'''
  Y=np.zeros((3))
  X = INT      #  S0,   I0 
  Y[0] = -beta * X[0] * X[1]
  Y[1] = beta*X[0]*X[1]  - gamma * X[1]
  Y[2] = gamma * X[1]
  return Y # for spicy.odeint

t_start =0.0 ; 
t_range = np.arange(t_start, t_end + t_inc, t_inc)
SIR= spi.odeint(simple_SIR, Input, t_range)

pl.figure(figsize=(15,8))
pl.plot(SIR[:, 0], '-g', label='Susceptibles')
pl.plot(SIR[:, 2], '-k', label='Recovereds')
pl.plot(SIR[:, 1], '-r', label='Infectious')
pl.legend(loc=0)
pl.title('Prediction of Simple nCOV-19 SIR model')
pl.xlabel('Time(day)')
pl.ylabel('individuals')
pl.show()

# beta 값을 가정해서 중국에서 연구된 논문에 나온 수치를 넣을것이라 신뢰되는 수치가 아닙니다. 타 논문에 의하면 Scipy annealing 기법을 통해서 구해지는데, 아직 제수준에는 코딩이 어렵네요. 도움이 필요합니다.

https://www.r-bloggers.com/sir-model-with-desolve-ggplot2/
R로도 풀어보려고 했으나 잘 안되서, R 프로그래밍을 잘하시는 분은 위 사이트를 참고해주세요.

# 2. 출생률과 사망률을 반영한  SIR 모델

# SIR model with births and deaths
![대체 텍스트](http://homepages.warwick.ac.uk/~masfz/ModelingInfectiousDiseases/Chapter2/Program_2.2/Equations.gif)

Parameters

    μ	is the per capita death rate, and the population level birth rate.
    β	is the transmission rate and incorporates the encounter rate between susceptible and infectious individuals together with the probability of transmission.
    γ	is called the removal or recovery rate, though often we are more interested in its reciprocal (1/γ) which determines the average infectious period.
    S(0)	is the initial proportion of the population that are susceptible.
    I(0)	is the initial proportion of the population that are infectious.All rates are specified in days.

Requirements.
All parameters must be positive, and S(0)+I(0) ≤ 1

In [ ]:
beta = 0.1934 # S ->I 감염율
gamma= 1/14 # I ->R 회복율 = 회복기간의 역수

nu = mu = 1/(70*365) # 자연사망율 반영

t_inc =1.0 ; t_end =150.0
'''
# Initial conditions
pop= 51780579 # 총인구수
test = 136707
test_ing = 28414
negative_tested = 102965 ; NT =negative_tested
print('positive_tested',test_ing + NT) # I0제외된 전체 test 받은 수

I0 = 5328 /test  # 3/3 기준 :확진자
S0 = ( test -I0 )/test # 3/3 기준 :양성판정+음성판정+검사중인 사람수( 총검사자 ) - 확진자'''

# 2/18 31번확진자 나온날 기준으로 초기 SIR 모델 만듦.
S0  = 9772 ; I0  = 31;R0 = 2

N = S0 + I0 + R0 
S0  = 9772 /N  # susceptible hosts
I0  = 31 /N    # infectious hosts
R0 = 2 /N      # recovered hosts
Input = (S0, I0, 0.0)

Input

In [ ]:
def simple_SIR(INT, t):
  '''The main set of equation'''
  Y=np.zeros((3))
  X = INT      #  S0,   I0 
  Y[0] = -beta * X[0] * X[1] - mu * X[0]
  Y[1] = beta*X[0]*X[1]  - gamma * X[1] - mu * X[1]
  Y[2] = gamma * X[1] - mu * X[2]                   # 자연사망자 제외 (위와 식이 조금 변형됨)
  return Y # for spicy.odeint

t_start =0.0 ; 
t_range = np.arange(t_start, t_end + t_inc, t_inc)
SIR= spi.odeint(simple_SIR, Input, t_range)

pl.figure(figsize=(15,8))
pl.plot(SIR[:, 0], '-g', label='Susceptibles')
pl.plot(SIR[:, 2], '-k', label='Recovereds')
pl.plot(SIR[:, 1], '-r', label='Infectious')
pl.legend(loc=0)
pl.title('Prediction of Simple nCOV-19 SIR model')
pl.xlabel('Time(day)')
pl.ylabel('individuals')
pl.show()

# SEIR model with 4 age classes and yearly aging

Key to this model are two basic assumptions:

1) Only individuals in the adult class give birth, and only individuals in the adult class die.
2) Births and deaths are continuous, but aging only happens once per year.

The continuous time dynamics are given by:![대체 텍스트](http://homepages.warwick.ac.uk/~masfz/ModelingInfectiousDiseases/Chapter3/Program_3.4/Equations.gif)

with the annual aging being given by:![대체 텍스트](http://homepages.warwick.ac.uk/~masfz/ModelingInfectiousDiseases/Chapter3/Program_3.4/Extra_Equations.gif)

    Parameters
    β	is the matrix of transmission rates and incorporates the encounter rate between susceptible and infectious individuals together with the probability of transmission.
    σ	is the rate at which individuals move from the exposed to the infectious classes. Its reciprocal (1/σ) is the average latent (exposed) period.
    γ	is called the removal or recovery rate, though often we are more interested in its reciprocal (1/γ) which determines the average infectious period.
    μ	is the death rate in each age group; it is assumed that only adults die.
    ν	is the birth rate into the childhood class; it is assumed only adults give birth.
    S(0)	is the vector of initial proportions of the population that are both susceptible and in a particular age group.
    E(0)	is the vector of initial proportions of the population that are both exposed and in a particular age group.
    I(0)	is the vector of initial proportions of the population that are both infectious and in a particular age group.All rates are specified in days.

Requirements.
All parameters must be positive, S(0) + E(0) + I(0) ≤ n  for each age group.

In [ ]:
import scipy.integrate
import numpy
import matplotlib.pyplot as plt

# Initial conditions
# 인구수 51780579
S0 = 9772 # :양성판정+검사중+음성판정
E0 = 818  # :검사중
I0 = 31   #  :확진자
R0 = 2    #  :완치자 + 사망자(0)


# Time vector
t = numpy.linspace(0,100,100)

N = S0 + I0 + R0 # 모집단

S0_ = S0/N
E0 =  E0/N
I0 = I0/N
R0 = R0/N

print(S0_)# 양성판정 확진자 + 음성판정 격리해지자수 비율 proporion

'''
Ro = 0.5 # 1인당 전파율 1월20일 보고된 한국코로나바이러스 역학조사 논문에 나온 수치)
print('논문에 보고된 Ro 평균',Ro)
To = 336 # 14*24 회복기간 2주  *  24시간
beta = (Ro/To) + (Ro/(To*S0))   # Ro 이용해서 beta 구하는 논문수식
print('\n논문 수식으로 구한 감염율 beta =',beta) 
논문 수식으로 구한 감염율 beta = 0.0014882475196382277
1월 초기에 전염병예측모델과 현재 현황이 많이 달라져서 사용할수 없음. 그당시 Ro, beta  모두 작은값.
'''
beta = 0.1934 # 중국논문에 나온 beta 값
ramda = 1/14
sigma = 0.25


Input = (S0_, E0 , I0)

def SEIR(INT, t):
  '''The main set of equation'''
  Y=np.zeros((3))
  X = INT      #  S0,   I0 
  Y[0] = mu -beta * X[0] * X[2] - mu *X[0]
  Y[1] = beta*X[0]*X[2]  - sigma * X[1] - mu * X[1]
  Y[2] = sigma * X[1] - gamma * X[2] - mu * X[2] #(자연사망자 제외)
  return Y # for spicy.odeint
  
  
t_start =0.0 ; t_end = 150 ; t_inc = 1.0
t_range = np.arange(t_start, t_end + t_inc, t_inc)
SEIR= spi.odeint(SEIR, Input, t_range)

Rec =1. - (SEIR[:,0]+ SEIR[:,1]+ SEIR[:,2])


pl.figure(figsize=(15,10))
pl.subplot(311)
pl.plot(SEIR[:,0], '-g', label='Susceptibles');pl.legend(loc=0)
pl.title('Prediction of nCOV-19 SEIR model')
pl.xlabel('Time(days)') # 국내 전염병의 추세가 하루하루 다르기 때문에.. 일주일단위보다는 1일단위로 보는게 맞는듯..
pl.ylabel('Susceptibles')

pl.subplot(312)
pl.plot(SEIR[:,1], '-b', label='Exposed')
pl.plot(SEIR[:,2], '-r', label='Infectious');pl.legend(loc=0)
pl.xlabel('Time(days)')
pl.ylabel('Infectious')

pl.subplot(313)
pl.plot(Rec, '-k', label='Recovereds')
pl.xlabel('Time(days)')
pl.legend(loc=0)
pl.ylabel('Recovereds')
pl.show()



In [ ]:

# 2/18 기준 확진자수로 그렸던 SIR 모델과 모습이 유사. 그당시 20일 지난 시점으로 보여짐.
#  3/3 기준 한국내 전염병 전파되서 확진자가 5천명을 넘음


# Initial conditions
# pop= 51780579 # 총인구수
test = 136707
test_ing = 28414
negative_tested = 102965 ; NT =negative_tested
print('positive_tested',test_ing + NT) # I0제외된 전체 test 받은 수


S0 = test 
E0 = test_ing
I0 = 5328
R0 = 41


# Time vector
t = numpy.linspace(0,100,100)

N = S0 + I0 + R0 # 모집단

S0_ = S0/N
I0 = I0/N
R0 = R0/N

beta= 0.1934

ramda = 1/14
gamma = 0.25

print('중국논문 beta ', beta)  



# ODEs 전염병예측모델에서 가장 전통적인 SIR 모델을 정의함
def SIR_model(y, t, beta, ramma):
    S, I, R = y
    
    dS_dt = -beta*S*I
    dI_dt = beta*S*I - ramma*I
    dR_dt = ramma*I
    
    return([dS_dt,dI_dt,dR_dt])
# Result
solution = scipy.integrate.odeint(SIR_model, [S0_,I0,R0], t, args=(beta, ramda))
solution = numpy.array(solution)

# plot result
plt.figure(figsize=(10,6))
plt.plot(t, solution[:, 0], label="S(t)")
plt.plot(t, solution[:, 1], label="I(t)")
plt.plot(t, solution[:, 2], label="R(t)")
plt.grid()
plt.legend()
plt.xlabel("Time")
plt.ylabel("Proportion")
plt.title("SIR model")
plt.box(False)
plt.show()